In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "notebooks"
    cd(joinpath(@__DIR__, "../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMolMonteCarlo`


In [2]:
folder = "assets/hpc_out/"
#folder = "assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, "_")[1]) ? parse(Int, split(file, "_")[1]) : max
    end
end

evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        @load "$folder$file" in_out_data
        id = parse(Int, split(file, "_")[1])
        string = "$(id): $(in_out_data.input.T) | $(in_out_data.input.ε) | $(in_out_data.output.αs[end]) | $(in_out_data.output.Es[1]), $(minimum(in_out_data.output.Es)), $(in_out_data.output.Es[end])"
        evaluation_strings[id] = string
    end
end

In [3]:
for elem in evaluation_strings
    println(elem)
end

1: 2.4 | 0.001 | 0.8211042 | 8097.730817128499, 8092.627806825541, 8097.578347268298
2: 2.4 | 0.001 | 0.77835 | 8097.730817128499, 8093.942421356985, 8105.521297545329
3: 2.4 | 0.001 | 0.82087094 | 8097.730817128499, 8092.188608572787, 8105.64794943632
4: 2.4 | 0.001 | 0.8283046 | 8110.120916987179, 8105.782670187582, 8140.524875935023
5: 2.4 | 0.001 | 0.82405376 | 8110.120916987179, 8104.954855019474, 8122.864026328485


In [4]:
function get_flat_realization(x, template_mol)
    n_mol = length(x) ÷ 6
    [(hvcat((n_mol), [exp(Rotations.RotationVecGenerator(x[i:i+2]...)) * template_mol .+ x[i+3:i+5] for i in 1:6:length(x)]...)...)...]
end

get_flat_realization (generic function with 1 method)

In [5]:
folder = "assets/hpc_out/"

for file in readdir(folder)
    try
        @load "$folder$file" in_out_data
        println("$folder$file")
        mindex = argmin(in_out_data.output.Es)
        state = in_out_data.output.states[mindex]
        id = parse(Int, split(file, "_")[1])
        radii = Base.Iterators.collect(Base.Iterators.flatten([in_out_data.input.template_radii for _ in 1:in_out_data.input.number_of_molecules]))
        MorphoMol.Utilities.state_to_poly(get_flat_realization(state, in_out_data.input.template_mol), radii, "$(folder)$(id)", in_out_data.input.number_of_molecules, length(in_out_data.input.template_radii))
    catch
        continue
    end
end

assets/hpc_out/1_6sae_10_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_11_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_12_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_13_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_1_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_2_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_3_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_4_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_5_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_6_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_7_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_8_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sae_9_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sag_1_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sag_2_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sag_4_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/1_6sag_5_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/2_6sae_10_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/2_6sae_12_protor_oj_0_0_os_0_85.jld2
assets/hpc_out/2_6sae_13_

In [6]:
function get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, subset_a::Vector{Int}, subset_b::Vector{Int})
    function transform_dist(template_centers, state, ss, idx)
        i = ss[1]
        j = ss[2]
        R1 = exp(RotationVecGenerator(state[(i-1) * 6 + 1:(i-1) * 6 + 3]...))
        T1 = state[(i-1) * 6 + 4:(i-1) * 6 + 6]
        R2 = exp(RotationVecGenerator(state[(j-1) * 6 + 1:(j-1) * 6 + 3]...))
        T2 = state[(j-1) * 6 + 4:(j-1) * 6 + 6]
        euclidean(T1 + R1 * template_centers[:,idx], T2 + R2 * template_centers[:,idx])
    end

    [abs(transform_dist(template_centers_a, state_a, subset_a, i) - transform_dist(template_centers_b, state_b, subset_b, i)) for i in 1:size(template_centers)[2]]
end

function sum_of_permutation(template_centers_a, template_centers_b, state_a, state_b, perm_a::Vector{Int}, perm_b::Vector{Int})
    n = size(template_centers)[2]
    d1 = sum(get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, perm_a[[1,2]], perm_b[[1,2]])) / n
    d2 = sum(get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, perm_a[[2,3]], perm_b[[2,3]])) / n
    d3 = sum(get_matched_distances_between_transformation_offsets(template_centers_a, template_centers_b, state_a, state_b, perm_a[[1,3]], perm_b[[1,3]])) / n
    println("$(d1), $(d2), $(d3)")
    (d1 + d2 + d3)/3.0
end

sum_of_permutation (generic function with 1 method)

In [7]:
@load "assets/hpc_assembly/3_6r7m_assembled_oj_0_0_os_0_85/49_6r7m_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

template_mol_exp, _ = MorphoMol.Utilities.poly_to_state("assets/input/poly/6r7m_protor.poly")
template_mol_exp = reshape(template_mol_exp,(3,Int(length(template_mol_exp) / 3)))

R1 = log(RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000]))
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = log(RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000]))
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = log(RotMatrix(@SMatrix[0.992567  0.121696  0.000000; -0.121696  0.992567  0.000000; 0.000000  0.000000  1.000000]))
T3 = @SVector[-19.48193, 22.01644, -46.53000]

state_exp_assembled = Vector{Float64}([])
for (R,T) in [(R1, T1), (R2, T2), (R3, T3)]
    state_exp_assembled = [state_exp_assembled; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end

permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(sum_of_permutation(template_mol_exp, template_mol_sim, state_exp_assembled, state_sim, [1, 2, 3], perm))
end

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/twZ5D/src/JLD2.jl:298


LoadError: SystemError: opening file "assets/hpc_assembly/3_6r7m_assembled_oj_0_0_os_0_85/49_6r7m_protor_oj_0_0_os_0_85.jld2": No such file or directory

In [58]:
@load "assets/hpc_assembly/3_6r7m_stacked_oj_0_0_os_0_85/43_6r7m_protor_oj_0_0_os_0_85.jld2" in_out_data
state_sim = in_out_data.output.states[argmin(in_out_data.output.Es)]
template_mol_sim = in_out_data.input.template_mol

template_mol_exp, _ = MorphoMol.Utilities.poly_to_state("assets/input/poly/6r7m_protor.poly")
template_mol_exp = reshape(template_mol_exp,(3,Int(length(template_mol_exp) / 3)))

R1 = log(RotMatrix(@SMatrix[0.628642  0.777695  0.000000; -0.777695  0.628642  0.000000; 0.000000  0.000000  1.000000]))
T1 = @SVector[-69.28043, 195.91352, -49.35000]

R2 = log(RotMatrix(@SMatrix[0.874450  0.485115  0.000000; -0.485115  0.874450  0.000000; 0.000000  0.000000  1.000000]))
T2 = @SVector[-61.30589, 104.11829, -47.94000]

R3 = log(RotMatrix(@SMatrix[0.803441  0.595384  0.000000; -0.595384  0.803441  0.000000; 0.000000  0.000000  1.000000]))
T3 = @SVector[-67.99970, 135.02619, -25.38000]


state_exp_assembled = Vector{Float64}([])
for (R,T) in [(R1, T1), (R2, T2), (R3, T3)]
    state_exp_assembled = [state_exp_assembled; [R[3,2], R[1,3], R[2,1], T[1], T[2], T[3]]]
end

permutations = [[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3,1,2], [3,2,1]]
for perm in permutations
    println(sum_of_permutation(template_mol_exp, template_mol_sim, state_exp_assembled, state_sim, [1, 2, 3], perm))
    println()
end

1.060546051243098, 1.7233861453834445, 2.704512138639089
1.829481445088544

3.8582220739511452, 1.7233861453834445, 7.206604684868613
4.262737634734401

1.060546051243098, 1.9544367320546299, 6.025618439072101
3.0135337407899434

5.358151778229962, 1.9544367320546299, 7.206604684868613
4.839731065051068

3.8582220739511452, 5.183991323542944, 6.025618439072101
5.02261061218873

5.358151778229962, 5.183991323542944, 2.704512138639089
4.415551746803998

